## Model Utils

In [ ]:
import os

from dotenv import load_dotenv
from openai import AzureOpenAI

load_dotenv()


client = AzureOpenAI(
    api_version="2024-12-01-preview",
    azure_endpoint="https://deepprompteastus2.openai.azure.com/",
    api_key=os.getenv("OPENAI_API_KEY", None),
)

gpt_41_deployment = "deepprompt-gpt-4.1-2025-04-14-global"
gpt_5_deployment = "deepprompt-gpt-5-2025-08-07-global"
gpt_5_mini_deployment = "deepprompt-gpt-5-mini-2025-08-07-global"
gpt_5_nano_deployment = "deepprompt-gpt-5-nano-2025-08-07-global"

In [ ]:
response = client.chat.completions.create(
    model=gpt_5_deployment,
    messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}],
    max_completion_tokens=8192,
)

print(response.choices[0].message.content)

<SEARCH>
      & $InstallerPath @LocalInstallerArguments
      if ($LASTEXITCODE -Ne "0") {
        $errMsg = "$ToolName installation failed"
</SEARCH>
<REPLACE>
      & $InstallerPath @LocalInstallerArguments
      if ($LASTEXITCODE -ne 0) {
        $errMsg = "$ToolName installation failed"
</REPLACE>


## Eval Utils

In [ ]:
import json

with open("./find_replace_benchmark.jsonl") as f:
    find_replace_bench = [json.loads(line) for line in f]

with open("./fully_rewrite_benchmark.jsonl") as f:
    fully_rewrite_bench = [json.loads(line) for line in f]

find_replace_sample = find_replace_bench[0]  # 'system_prompt', 'user_prompt', 'ground_truth', 'metadata'

system_prompt = find_replace_sample["system_prompt"]
user_prompt = find_replace_sample["user_prompt"]
ground_truth = find_replace_sample["ground_truth"]


In [ ]:
response = client.chat.completions.create(
    model=gpt_5_deployment,
    messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}],
    max_completion_tokens=8192,
)

print(response.choices[0].message.content)

<SEARCH>
      & $InstallerPath @LocalInstallerArguments
      if ($LASTEXITCODE -Ne "0") {
        $errMsg = "$ToolName installation failed"
</SEARCH>
<REPLACE>
      & $InstallerPath @LocalInstallerArguments
      if ($LASTEXITCODE -ne 0) {
        $errMsg = "$ToolName installation failed"
</REPLACE>


In [ ]:
from utils import apply_diff

original_code = find_replace_bench[0]["original_code"]
modifications = """
<SEARCH>
      & $InstallerPath @LocalInstallerArguments
      if ($LASTEXITCODE -Ne "0") {
        $errMsg = "$ToolName installation failed"
</SEARCH>
<REPLACE>
      & $InstallerPath @LocalInstallerArguments
      if ($LASTEXITCODE -ne 0) {
        $errMsg = "$ToolName installation failed"
</REPLACE>
"""

_, modified_code = apply_diff(original_code, modifications)

print(modified_code == ground_truth)

False


### Share

## Evaluation

### GPT

### Claude

In [4]:
import os

import anthropic
from dotenv import load_dotenv

load_dotenv()

anthropic_api_key = os.getenv("ANTHROPIC_API_KEY", None)

claude_client = anthropic.Anthropic(api_key=anthropic_api_key)

response = claude_client.messages.create(
    model="claude-sonnet-4-0",
    max_tokens=8192,
    messages=[{"role": "user", "content": "who are you?"}],
)

print(response.content)

BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Anthropic API. Please go to Plans & Billing to upgrade or purchase credits.'}, 'request_id': 'req_011CSKzocQEPWHiauhC7GoV5'}

### Qwen3-4B

In [7]:
import json

from utils import apply_diff

with open("results/find_replace_gpt-5_results.jsonl") as f:
    results = [json.loads(line) for line in f]

results[0]


success, modified_code = apply_diff(results[0]["original_code"], results[0]["model_response"])

print(success)
print(modified_code)

True
<#
.SYNOPSIS
Entry point script for installing native tools

.DESCRIPTION
Reads $RepoRoot\\global.json file to determine native assets to install
and executes installers for those tools

.PARAMETER BaseUri
Base file directory or Url from which to acquire tool archives

.PARAMETER InstallDirectory
Directory to install native toolset.  This is a command-line override for the default
Install directory precedence order:
- InstallDirectory command-line override
- NETCOREENG_INSTALL_DIRECTORY environment variable
- (default) %USERPROFILE%/.netcoreeng/native

.PARAMETER Clean
Switch specifying to not install anything, but cleanup native asset folders

.PARAMETER Force
Clean and then install tools

.PARAMETER DownloadRetries
Total number of retry attempts

.PARAMETER RetryWaitTimeInSeconds
Wait time between retry attempts in seconds

.PARAMETER GlobalJsonFile
File path to global.json file

.NOTES
#>
[CmdletBinding(PositionalBinding=$false)]
Param (
  [string] $BaseUri = "https://netcorena